# TO RUN
1. Make sure you installed all the dependencies
2. open **'init_notebook.py'** and change the **projectFolder** to the src folder of the repository.
3. In the following cell change the dataDir to the data directory of the InD dataset

In [1]:
dataDir = "G:AV datasets/inD-dataset-v1.0/data/"
dataDir = "E:/Datasets/inD-dataset-v1.0/data/"

In [2]:
exec(open('init_notebook.py').read())
%load_ext autoreload
%autoreload 2
%tb

current working dirD:\AV\DataSetExploration\drone-dataset-tools\src


No traceback available to show.


In [3]:
import logging
logging.basicConfig(level=logging.INFO)
from loguru import logger
logger.remove()
logger.add(sys.stdout, format="{level} {message}", filter="my_module", level="INFO")

from extractors.loader import Loader
from tools.TrajectoryUtils import TrajectoryUtils

In [4]:
loader = Loader(dataDir, 'inD')

In [5]:
loc2data = loader.getLocationData(2, useSceneConfigToExtract=True, precomputeSceneData=False)

INFO:root:recordingIds: [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
INFO:root:Downsampling recording 18 from 25.0 to 5
downsampling: 100%|█████████████████████████████████████████████████████████████| 625989/625989 [00:26<00:00, 23709.28it/s]
INFO:root:Downsampling recording 19 from 25.0 to 5
downsampling: 100%|█████████████████████████████████████████████████████████████| 492008/492008 [00:20<00:00, 23739.14it/s]
INFO:root:Downsampling recording 20 from 25.0 to 5
downsampling: 100%|█████████████████████████████████████████████████████████████| 634916/634916 [00:26<00:00, 23598.18it/s]
INFO:root:Downsampling recording 21 from 25.0 to 5
downsampling: 100%|█████████████████████████████████████████████████████████████| 796178/796178 [00:33<00:00, 23879.52it/s]
INFO:root:Downsampling recording 22 from 25.0 to 5
downsampling: 100%|█████████████████████████████████████████████████████████████| 753743/753743 [00:31<00:00, 23908.96it/s]
INFO:root:Downsampling recording 23 from 25.0 to 5
d

In [ ]:
sceneId = 21
sceneConfig = {
      "centerXPixel": 302.5,
      "centerYPixel": 345.5,
      "centerX": 29.57,
      "centerY": -33.77,
      "angle": 16.0,
      "roadWidth": 9.0,
      "boxWidth": 15.0
    }
boxWidth = sceneConfig["boxWidth"]
roadWidth = sceneConfig["roadWidth"]
pedData = loc2data.getSceneCrossingDf(sceneId, boxWidth, roadWidth)
pedData.groupby(["uniqueTrackId"]).count()

In [7]:
otherData = loc2data.getSceneOtherDf(sceneId)

recording-18-scene10-bicycle-Ids: 100%|████████████████████████████████████████████████████| 92/92 [00:02<00:00, 43.23it/s]
recording-18-scene10-truck_bus-Ids: 0it [00:00, ?it/s]
recording-18-scene11-bicycle-Ids: 100%|████████████████████████████████████████████████████| 92/92 [00:02<00:00, 44.80it/s]
recording-18-scene11-truck_bus-Ids: 0it [00:00, ?it/s]
recording-18-scene20-bicycle-Ids: 100%|████████████████████████████████████████████████████| 92/92 [00:02<00:00, 44.35it/s]
recording-18-scene20-truck_bus-Ids: 0it [00:00, ?it/s]
recording-18-scene21-bicycle-Ids: 100%|████████████████████████████████████████████████████| 92/92 [00:02<00:00, 43.79it/s]
recording-18-scene21-truck_bus-Ids: 0it [00:00, ?it/s]
recording-18-scene22-bicycle-Ids: 100%|████████████████████████████████████████████████████| 92/92 [00:02<00:00, 44.25it/s]
recording-18-scene22-truck_bus-Ids: 0it [00:00, ?it/s]
recording-18-scene30-bicycle-Ids: 100%|████████████████████████████████████████████████████| 92/92 [00:02

In [9]:
from extractors.SceneData import SceneData
sceneData = SceneData(
    2,
    0.00814636091724502,
    sceneId,
    sceneConfig,
    boxWidth,
    roadWidth,
    pedData=pedData,
    otherData=otherData,
    backgroundImagePath="G:AV datasets/inD-dataset-v1.0/data/18_background.png"
)

sceneData.buildLocalInformation()
# print(sceneData.warnings)

clipping ped trajectories for scene # 21 with width offset 4.0: 100%|██████████████████████| 70/70 [00:01<00:00, 42.01it/s]
clipping other trajectories for scene # 21: 100%|██████████████████████████████████████| 2287/2287 [01:31<00:00, 25.07it/s]
INFO:root:adding dynamics for scene 21
clipping ped trajectories for scene # 21 with width offset 0.0: 100%|█████████████████████| 70/70 [00:00<00:00, 760.87it/s]


TypeError: unsupported operand type(s) for -: 'set' and 'list'

In [ ]:
print(sceneData.problematicIds)

In [ ]:
# trackId = 25117
localPedData = sceneData.getPedDataInSceneCoordinates()
# localPedData[localPedData["uniqueTrackId"]==trackId]
localPedData.groupby(["uniqueTrackId"]).size()

In [ ]:
localPedData.head()

In [ ]:

localOtherData = sceneData.getOtherDataInSceneCoordinates()
# localPedData[localPedData["uniqueTrackId"]==trackId]
localOtherData.groupby(["uniqueTrackId"]).size()

In [ ]:
# pedData[(pedData["uniqueTrackId"]==trackId)]

In [ ]:

# pedData.groupby(["uniqueTrackId"]).count()

In [ ]:
from tools.TrajectoryUtils import TrajectoryUtils
from tools.TrajectoryVisualizer import TrajectoryVisualizer
import matplotlib.pyplot as plt
import numpy as np
visualizer = TrajectoryVisualizer(loader)

In [ ]:
# sceneData.sceneConfig
# sceneData.orthoPxToMeter

In [ ]:
visualizer.showLocationSceneData(sceneData, onlyClipped=False, showLocal=False, showOthers=True, ids=[25538, 25117])

In [ ]:
visualizer.showLocationSceneData(sceneData, onlyClipped=True, showLocal=False, showOthers=True, ids=[25538, 25117])

In [ ]:
visualizer.showLocationSceneData(sceneData, onlyClipped=False, showLocal=False, showOthers=True, ids=[25117])

In [ ]:
ped25117 = pedData[pedData["uniqueTrackId"]==25117]

In [ ]:
ped25117.to_csv("../data/ped25117.csv")

In [ ]:
# visualize all peds
from math import ceil
allPedIds = sceneData.uniquePedIds()
limit = 10
nPedIds = len(allPedIds)
nPages = ceil(nPedIds / limit)
for page in range(nPages):
    offset = page * limit
    visualizer.showLocationSceneData(sceneData, onlyClipped=False, showLocal=False, showOthers=False, offset=offset, limit=limit)